In [72]:
import pandas as pd
import numpy as np

In [15]:
matched_blotter = pd.read_csv('./outputs/matched_blotter.csv')

In [22]:
matched_blotter['Exit Date'].iloc[-1]

'2024-03-21'

In [80]:
matched_blotter['Risk Free Rate'].astype(float).mean()

2.210537634408602

In [102]:
start_date = matched_blotter['Exit Date'].iloc[1]
exit_date = matched_blotter['Exit Date'].iloc[-1]
start_date, exit_date

initial_capital = 1000000
end_capital = initial_capital
for ret in matched_blotter['Return']:
    end_capital *= (1 + ret / 100)

# Total Nominal Return
total_nominal_return = end_capital - initial_capital

# Calculate Geometric Mean Return Per Month
months = (pd.to_datetime(exit_date) - pd.to_datetime(start_date)).days / 30
monthly_returns = (end_capital / initial_capital)**(1 / months) - 1

# Calculate Sharpe Ratio
average_daily_return = matched_blotter['Return per Trade Day'].mean()
std_dev_daily_returns = matched_blotter['Return per Trade Day'].std()
average_risk_free_rate = matched_blotter['Risk Free Rate'].astype(float).mean() / 100
sharpe_ratio = (average_daily_return - average_risk_free_rate) / std_dev_daily_returns

# Create the portfolio_summary DataFrame
portfolio_summary = pd.DataFrame({
    'Start Date': [start_date],
    'End Date': [exit_date],
    'Initial Capital': [initial_capital],
    'End Capital': [end_capital],
    'Total Nominal Return': [total_nominal_return],
    'Geometric Mean Return': [monthly_returns * 100],
    'Sharpe Ratio': [sharpe_ratio]
})

portfolio_summary.to_csv('./outputs/portfolio_summary.csv', index=False)

In [103]:
portfolio_summary

,Start Date,End Date,Initial Capital,End Capital,Total Nominal Return,Geometric Mean Return,Sharpe Ratio
0,2021-04-23,2024-03-21,1000000,2.388467e+06,1.388467e+06,2.487592,0.233084


In [88]:
import yfinance as yf

# Download SPY data
spy = yf.download('SPY')

# Calculate the period return
spy['Period Return'] = spy['Close'].pct_change()

Index(['Entry Date', 'Symbol', 'Quantity', 'Entry Price', 'Exit Date',
       'Exit Price', 'Return', 'Return per Trade Day', 'Return (USD)',
       'Risk Free Rate'],
      dtype='object')

In [123]:
# Initial capital
initial_capital = 1000000

# Calculating Portfolio Profit/Loss
matched_blotter['Portfolio Profit Loss'] = matched_blotter['Quantity'] * (matched_blotter['Exit Price'] - matched_blotter['Entry Price'])

# Initializing Total Capital and Period Return lists
total_capital = initial_capital
capital_list = []
period_return_list = []

# Calculating Total Capital and Period Return for each period
for i, row in matched_blotter.iterrows():
    total_capital += row['Portfolio Profit Loss']
    capital_list.append(total_capital)
    if i == 0:
        period_return_list.append(row['Portfolio Profit Loss'] / initial_capital * 100)
    else:
        period_return_list.append(row['Portfolio Profit Loss'] / (capital_list[-2]) * 100)

# Creating new DataFrame
portfolio_performance = pd.DataFrame({
    'Date': matched_blotter['Exit Date'],
    'Portfolio Profit Loss': matched_blotter['Portfolio Profit Loss'],
    'Total Capital': capital_list,
    'Period Return (%)': period_return_list
})

portfolio_performance['Period Return (%)'] = portfolio_performance['Period Return (%)'] * 100
portfolio_performance['SPY Period Return (%)'] = portfolio_performance['Date'].map(spy['Period Return'])
portfolio_performance['SPY Period Return (%)'].bfill(inplace=True)
portfolio_performance['SPY Period Return (%)'] = portfolio_performance['SPY Period Return (%)'] * 100

portfolio_performance.to_csv('./outputs/portfolio_performance.csv', index=False)

portfolio_performance

,Date,Portfolio Profit Loss,Total Capital,Period Return (%),SPY Period Return (%)
0,2021-04-19,1386.705948,1.001387e+06,13.867059,-0.491305
1,2021-04-23,1373.411244,1.002760e+06,13.715094,1.084241
2,2021-05-02,916.050376,1.003676e+06,9.135289,-0.616934
3,2021-05-04,-697.556008,1.002979e+06,-6.950011,-0.616934
4,2021-05-13,-343.747027,1.002635e+06,-3.427262,1.201252
...,...,...,...,...,...
88,2024-02-07,568.361531,1.062158e+06,5.353875,0.834041
89,2024-02-19,229.511844,1.062387e+06,2.160808,-0.994276
90,2024-02-25,-159.022737,1.062228e+06,-1.496844,-0.994276
91,2024-03-15,4025.563478,1.066254e+06,37.897358,-0.994276


In [14]:
matched_blotter.drop(columns=['Instrument'], inplace=True)
matched_blotter.to_csv('./outputs/matched_blotter.csv', index=False)

In [8]:
unmatched_blotter_columns = [
    'Instrument', 
    'Close Price',
    'Date',
    'Trip',
    'Quantity'
]

unmatched_blotter = pd.DataFrame()

for index, row in matched_blotter.iterrows():
    entry_date = row['Entry Date']
    symbol = row['Symbol']
    quantity = row['Quantity']
    entry_price = row['Entry Price']
    exit_date = row['Exit Date']
    return_value = row['Return']

    entry_row = ['SOL', entry_price, entry_date, 'Entry', quantity]
    exit_row = ['SOL', entry_price, exit_date, 'Exit', quantity]
    
    entry_series = pd.Series(entry_row, index=unmatched_blotter_columns)
    exit_series = pd.Series(exit_row, index=unmatched_blotter_columns)
    
    unmatched_blotter = pd.concat([unmatched_blotter, entry_series.to_frame().T, exit_series.to_frame().T], ignore_index=True)

unmatched_blotter.to_csv('./outputs/unmatched_blotter.csv', index=False)

In [12]:
matched_blotter

,Entry Date,Symbol,Quantity,Entry Price,Exit Date,Exit Price,Return,Return per Trade Day,Return (USD),Instrument
0,2021-04-15,SOL-USD,359.475421,27.818314,2021-04-19,31.675896,3.246536,3.246536,1386.705948,SOL
1,2021-04-21,SOL-USD,307.568878,32.513042,2021-04-23,36.978420,6.434660,6.434660,1373.411244,SOL
2,2021-04-30,SOL-USD,232.796343,42.956001,2021-05-02,46.890987,4.382456,4.382456,916.050376,SOL
3,2021-05-03,SOL-USD,217.705135,45.933689,2021-05-04,42.729557,-7.230793,-7.230793,-697.556008,SOL
4,2021-05-12,SOL-USD,235.530703,42.457310,2021-05-13,40.997852,-3.497941,-3.497941,-343.747027,SOL
...,...,...,...,...,...,...,...,...,...,...
88,2024-02-05,SOL-USD,104.658715,95.548660,2024-02-07,100.979279,2.763984,2.763984,568.361531,SOL
89,2024-02-17,SOL-USD,91.756574,108.984016,2024-02-19,111.485329,1.134588,1.134588,229.511844,SOL
90,2024-02-21,SOL-USD,95.124651,105.125221,2024-02-25,103.453491,-0.400752,-0.400752,-159.022737,SOL
91,2024-03-03,SOL-USD,76.657214,130.450867,2024-03-15,182.964691,2.819138,2.819138,4025.563478,SOL


In [62]:
matched_blotter['Entry Date']
output_df.sort_index(inplace=True)

In [56]:
matched_blotter['Risk Free Rate'] = matched_blotter['Entry Date'].map(output_df['COUPON EQUIVALENT'])

In [68]:
matched_blotter['Risk Free Rate'].bfill(inplace=True)

In [69]:
matched_blotter

,Entry Date,Symbol,Quantity,Entry Price,Exit Date,Exit Price,Return,Return per Trade Day,Return (USD),Risk Free Rate
0,2021-04-15,SOL-USD,359.475421,27.818314,2021-04-19,31.675896,3.246536,3.246536,1386.705948,0.02
1,2021-04-21,SOL-USD,307.568878,32.513042,2021-04-23,36.978420,6.434660,6.434660,1373.411244,0.03
2,2021-04-30,SOL-USD,232.796343,42.956001,2021-05-02,46.890987,4.382456,4.382456,916.050376,0.01
3,2021-05-03,SOL-USD,217.705135,45.933689,2021-05-04,42.729557,-7.230793,-7.230793,-697.556008,0.04
4,2021-05-12,SOL-USD,235.530703,42.457310,2021-05-13,40.997852,-3.497941,-3.497941,-343.747027,0.02
...,...,...,...,...,...,...,...,...,...,...
88,2024-02-05,SOL-USD,104.658715,95.548660,2024-02-07,100.979279,2.763984,2.763984,568.361531,5.38
89,2024-02-17,SOL-USD,91.756574,108.984016,2024-02-19,111.485329,1.134588,1.134588,229.511844,5.40
90,2024-02-21,SOL-USD,95.124651,105.125221,2024-02-25,103.453491,-0.400752,-0.400752,-159.022737,5.40
91,2024-03-03,SOL-USD,76.657214,130.450867,2024-03-15,182.964691,2.819138,2.819138,4025.563478,5.38


In [124]:
df = pd.read_csv('./data/model_data.csv')

In [136]:
df['uniswap_predicted'].min()

4.201882763100727

In [ ]:
import pickle


x_input = tensor([bitcoin_predicted_price, raydium_predicted_price, raydium_tvl, ethereum_predicted_price, uniswap_predicted_price])

with open('./models/mlp.pkl', 'rb') as f:
    mlp_model = pickle.load(f)


mlp_model
#MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1500, random_state=42)

array([124.5945681])

In [139]:
mlp_model

MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1500, random_state=42)